# Tutorial 2 - Shared Variables and Global Context

- This is a very important feature of TaskGen, as it helps with learning through tasks and dynamic adaptation of the Agent with changing environments
- `Shared Variables` help to keep track of persistent states for the agent
- Global Context can provide some of the `Shared Variables` as a global context to the Agent

# Setup Guide

## Step 1: Install TaskGen

In [1]:
# !pip install taskgen-ai

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from taskgen import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string
- Note: If you do not define your own LLM, OpenAI 'gpt-3.5-turbo' will be used as default

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Sentiment: Positive'

# 1. Shared Variables

*"Because text is not enough"* - Anonymous

- `shared_variables` is a dictionary, that is initialised in Agent (default empty dictionary), and can be referenced by any function of the agent (including Inner Agents and their functions)
- This can be useful for non-text modalitiies (e.g. audio, pdfs, image) and lengthy text modalities, which we do not want to output into `subtasks_completed` directly
- To use, simply define an External Function with `shared_variables` as the first input variable, from which you can access and modify `shared_variables` directly
- The agent will also be able to be self-referenced in the External Function via `shared_variables['agent']`, so you can change the agent's internal parameters via `shared_variables`
- If the function has no output because the output is stored in `shared_variables`, the default return value will be `{'Status': 'Completed'}`

### Example External Function using `shared_variables`
```python
# Use shared_variables as input to your external function to access and modify the shared variables
def generate_quotes(shared_variables, number_of_quotes: int, category: str):
    ''' Generates number_of_quotes quotes about category '''
    # Retrieve from shared variables
    my_quote_list = shared_variables['Quote List']
    
    # Generate the quotes
    res = strict_json(system_prompt = f'''Generate {number_of_quotes} sentences about {category}. 
Do them in the format "<Quote> - <Person>", e.g. "The way to get started is to quit talking and begin doing. - Walt Disney"
Ensure your quotes contain only ' within the quote, and are enclosed by " ''',
                      user_prompt = '',
                      output_format = {'Quote List': f'list of {number_of_quotes} quotes, type: List[str]'},
                      llm = llm)
    
    my_quote_list.extend([f'Category: {category}. '+ x for x in res['Quote List']])
    
    # Store back to shared variables
    shared_variables['Quote List'] = my_quote_list
```

In [5]:
# Use shared_variables as input to your external function to access and modify the shared variables
def generate_quotes(shared_variables, number_of_quotes: int, category: str):
    ''' Generates number_of_quotes quotes about category '''
    # Retrieve from shared variables
    my_quote_list = shared_variables['Quote List']
    
    # Generate the quotes
    res = strict_json(system_prompt = f'''Generate {number_of_quotes} sentences about {category}. 
Do them in the format "<Quote> - <Person>", e.g. "The way to get started is to quit talking and begin doing. - Walt Disney"
Ensure your quotes contain only ' within the quote, and are enclosed by " ''',
                      user_prompt = '',
                      output_format = {'Quote List': f'list of {number_of_quotes} quotes, type: List[str]'},
                      llm = llm)
    
    my_quote_list.extend([f'Category: {category}. '+ x for x in res['Quote List']])
    
    # Store back to shared variables
    shared_variables['Quote List'] = my_quote_list

In [6]:
# Define the quote generator agent and the shared_variables - Note the naming convention of s_ at the start of the names for shared variables
my_agent = Agent('Quote Generator', 'Generates Quotes according to category',
                 default_to_llm = False, # do not provide llm as a default function to Agent to prevent hallucinations
                 shared_variables = {'Quote List': []},
                 llm = llm).assign_functions([generate_quotes])

In [7]:
output = my_agent.run('Generate three quotes about life')

Observation: No subtasks have been completed yet for the assigned task.
Thoughts: To complete the assigned task, the agent needs to generate three quotes about life.
Subtask identified: Use the 'generate_quotes' function to generate three quotes about life.
Calling function generate_quotes with parameters {'number_of_quotes': 3, 'category': 'life'}
> {'Status': 'Completed'}

Observation: The subtask to generate three quotes about life has been completed successfully.
Thoughts: Since the subtask has been completed, the next step is to pass the final output to the user using the 'end_task' function.
Subtask identified: End Task
Task completed successfully!



In [8]:
# visualise quote list
print('Quote List:', my_agent.shared_variables['Quote List'])

Quote List: ["Category: life. Life is what happens when you're busy making other plans. - John Lennon", "Category: life. In the end, it's not the years in your life that count. It's the life in your years. - Abraham Lincoln", 'Category: life. The purpose of our lives is to be happy. - Dalai Lama']


In [9]:
my_agent.reset() # always reset agent if the task is something new to prevent misinterpretation
output = my_agent.run('Generate three quotes about happiness')

Observation: No subtasks have been completed yet for the assigned task.
Thoughts: To complete the assigned task, the agent needs to generate three quotes about happiness.
Subtask identified: Use the 'generate_quotes' function with the input values '3' for 'number_of_quotes' and 'happiness' for 'category' to generate the required quotes.
Calling function generate_quotes with parameters {'number_of_quotes': 3, 'category': 'happiness'}
> {'Status': 'Completed'}

Observation: The subtask to generate three quotes about happiness has been successfully completed.
Thoughts: Since the subtask has been completed, the next step is to pass the final output to the user using the 'end_task' function.
Subtask identified: End Task
Task completed successfully!



In [10]:
# visualise quote list
print('Quote List:', my_agent.shared_variables['Quote List'])

Quote List: ["Category: life. Life is what happens when you're busy making other plans. - John Lennon", "Category: life. In the end, it's not the years in your life that count. It's the life in your years. - Abraham Lincoln", 'Category: life. The purpose of our lives is to be happy. - Dalai Lama', 'Category: happiness. Happiness is not something ready made. It comes from your own actions. - Dalai Lama', 'Category: happiness. The only thing that will make you happy is being happy with who you are. - Goldie Hawn', 'Category: happiness. Happiness is a choice. You can choose to be happy. - Valerie Bertinelli']


# 2. Global Context

- `Global Context` is a very powerful feature in TaskGen, as it allows the Agent to be updated with the latest environmental state before every decision it makes
- It also allows for learnings in `shared_variables` to be carried across tasks, making the Agent teachable and learn through experiences
- A recommended practice is to always store the learnings of the Agent during the External Function call, and reset the Agent after each task, so that `subtasks_completed` will be as short as possible to avoid confusion to the Agent

- There are two ways to use `Global Context`, and both can be used concurrently:
    - 1. `global_context`
        - If all you need in the global context is `shared_variables` without any modification to it, then you can use `global_context`
        - `global_context` is a string with `<shared_variables_name>` enclosed with `<>`. These <> will be replaced with the actual variable in `shared_variables`
    - 2. `get_global_context` 
        - `get_global_context` is a function that takes in the agent's internal parameters (self) and outputs a string to the LLM to append to the prompts of any LLM-based calls internally, e.g. `get_next_subtask`, `use_llm`, `reply_to_user`
    - You have full flexibility to access anything the agent knows and process the `shared_variables` as required and configure a global prompt to the agent
    

# Example for `global_context` : Inventory Manager
- We can use `Global Context` to keep track of inventory state
- We simply get the functions `add_item_to_inventory` and `remove_item_from_inventory` to modify the `shared_variable` named `Inventory`
- Note we can also put rule-based checks like checking if item is in inventory before removing inside the function
- Even after task reset, the Agent still knows the inventory because of `Global Context`

```python
def add_item_to_inventory(shared_variables, item: str) -> str:
    ''' Adds item to inventory, and returns outcome of action '''
    shared_variables['Inventory'].append(item)
    return f'{item} successfully added to Inventory'
    
def remove_item_from_inventory(shared_variables, item: str) -> str:
    ''' Removes item from inventory and returns outcome of action '''
    if item in shared_variables['Inventory']:
        shared_variables['Inventory'].remove(item)
        return f'{item} successfully removed from Inventory'
    else:
        return f'{item} not found in Inventory, unable to remove'
    
agent = Agent('Inventory Manager', 
              'Adds and removes items in Inventory. Only able to remove items if present in Inventory',
              shared_variables = {'Inventory': []},
              global_context = 'Inventory: <Inventory>', # Add in Global Context here with shared_variables Inventory
              llm = llm).assign_functions([add_item_to_inventory, remove_item_from_inventory])
```

In [11]:
def add_item_to_inventory(shared_variables, item: str) -> str:
    ''' Adds item to inventory, and returns outcome of action '''
    shared_variables['Inventory'].append(item)
    return f'{item} successfully added to Inventory'
    
def remove_item_from_inventory(shared_variables, item: str) -> str:
    ''' Removes item from inventory and returns outcome of action '''
    if item in shared_variables['Inventory']:
        shared_variables['Inventory'].remove(item)
        return f'{item} successfully removed from Inventory'
    else:
        return f'{item} not found in Inventory, unable to remove'
    
agent = Agent('Inventory Manager', 
              'Adds and removes items in Inventory. Only able to remove items if present in Inventory',
              shared_variables = {'Inventory': []},
              global_context = 'Inventory: <Inventory>', # Add in Global Context here with shared_variables Inventory
              llm = llm).assign_functions([add_item_to_inventory, remove_item_from_inventory])

In [12]:
output = agent.run('Add apples and oranges')

Observation: No items have been added to the Inventory yet.
Thoughts: Since no items have been added to the Inventory, the first step is to add the items 'apples' and 'oranges' to the Inventory.
Subtask identified: Add 'apples' and 'oranges' to the Inventory.
Calling function add_item_to_inventory with parameters {'item': 'apples'}
> {'output_1': 'apples successfully added to Inventory'}

Observation: One item 'apples' has been successfully added to the Inventory. The task involves adding 'oranges' to the Inventory.
Thoughts: Since 'apples' have been added successfully, the focus should be on adding 'oranges' to the Inventory. No need to remove any items as per the task requirements.
Subtask identified: Add oranges to the Inventory.
Calling function add_item_to_inventory with parameters {'item': 'oranges'}
> {'output_1': 'oranges successfully added to Inventory'}

Observation: Items 'apples' and 'oranges' have been successfully added to the Inventory.
Thoughts: Since the items 'apples'

In [13]:
# visualise the inventory - we should have both apples and oranges
agent.shared_variables['Inventory']

['apples', 'oranges']

In [14]:
# even after agent reset, we will still know the Inventory because of global context
agent.reset()
output = agent.run('Remove only apples')

Observation: Inventory Manager can only remove items that are present in the Inventory. The Inventory currently contains apples and oranges.
Thoughts: To complete the remainder of the Assigned Task, the Inventory Manager should focus on removing only the apples from the Inventory.
Subtask identified: Remove the item 'apples' from the Inventory.
Calling function remove_item_from_inventory with parameters {'item': 'apples'}
> {'output_1': 'apples successfully removed from Inventory'}

Observation: The task of removing apples has been completed successfully by removing the item "apples" from the Inventory.
Thoughts: Since the task of removing apples has been completed, the next step is to end the task as it has been successfully executed.
Subtask identified: End Task
Task completed successfully!



In [15]:
# visualise the inventory - we should only have oranges
agent.shared_variables['Inventory']

['oranges']

In [16]:
# even after agent reset, we will still know the Inventory because of global context
# Here we will know that pears are not part of inventory and cannot be removed
agent.reset()
output = agent.run('Remove only pears')

Observation: No subtasks have been completed yet for the assigned task.
Thoughts: To complete the assigned task, we need to check if 'pears' are present in the Inventory and then remove them if they exist.
Subtask identified: Check if 'pears' are present in the Inventory and remove them if they exist.
Calling function remove_item_from_inventory with parameters {'item': 'pears'}
> {'output_1': 'pears not found in Inventory, unable to remove'}

Observation: The attempt to remove pears from the Inventory failed as pears were not found in the Inventory.
Thoughts: Since pears are not in the Inventory, there is no need to attempt to remove them again. It would be more efficient to focus on other tasks.
Subtask identified: End Task
Task completed successfully!



In [17]:
# visualise the inventory - we should only have oranges
agent.shared_variables['Inventory']

['oranges']

# Advanced Example for `get_global_context`: Maze Navigator
- We can use the global context to let agent know present state in a 2D grid, and obstacle positions that the agent has seen
- We use the `get_global_context` function as we want to configure more details about how we use the `shared_variables`
- Task: Given current position and end position, navigate to end position using actions: Up, Down, Left, Right, Stay

In [18]:
# These are the utility functions
def generate_grid(size, start_pos, exit_pos, obstacles):
    '''Generates a grid with obstacles'''
    grid = [[' ' for _ in range(size)] for _ in range(size)]
    grid[start_pos[0]][start_pos[1]] = 'S'  # Start
    grid[exit_pos[0]][exit_pos[1]] = 'E'  # Exit
    
    # Mark a basic path (optional, for simplicity in ensuring a path)
    # This part could be removed or replaced with a more sophisticated path marking
    path = set()
    for i in range(min(start_pos[0], exit_pos[0]), max(start_pos[0], exit_pos[0]) + 1):
        path.add((i, start_pos[1]))
    for j in range(min(start_pos[1], exit_pos[1]), max(start_pos[1], exit_pos[1]) + 1):
        path.add((exit_pos[0], j))
    
    # Randomly add obstacles
    count = 0
    while count < obstacles:
        r, c = random.randint(0, size-1), random.randint(0, size-1)
        if (r, c) not in path and grid[r][c] != 'O' and (r, c) != start_pos and (r, c) != exit_pos:
            grid[r][c] = 'O'
            count += 1
            
    return grid

def print_grid(grid):
    '''Prints the grid'''
    for row in grid:
        print(' '.join(row))
        
def check_valid_moves(cur_pos, grid, grid_size):
    '''Checks the valid moves in the grid given the cur_pos and grid_size. Returns list of valid moves within action space of Up, Down, Left, Right, Stay'''
    row, col = cur_pos
    mapping = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1), 'Stay': (0, 0)}
    valid_moves = []
    for key, value in mapping.items():
        row_offset, col_offset = value
        # check if valid move
        if 0 <= row+row_offset < grid_size and 0 <= col+col_offset < grid_size and grid[row+row_offset][col+col_offset] != 'O':
            valid_moves.append(key)
    return valid_moves

def update_obstacles(cur_pos, grid, grid_size, known_obstacle_pos):
    '''Returns the updated known obstacle positions in the current grid given the cur_pos and grid_size'''
    row, col = cur_pos
    mapping = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1), 'Stay': (0, 0)}
    for key, value in mapping.items():
        row_offset, col_offset = value
        next_row, next_col = row+row_offset, col+col_offset
        # check if valid move
        if 0 <= next_row < grid_size and 0 <= next_col < grid_size:
            # adds in obstacle if observed
            if grid[next_row][next_col] == 'O' and (next_row, next_col) not in known_obstacle_pos:
                known_obstacle_pos.append((next_row, next_col))
            # remove obstacle that is not observed
            elif (next_row, next_col) in known_obstacle_pos:
                known_obstacle_pos.remove((next_row, next_col))
    return known_obstacle_pos

In [19]:
def move(shared_variables, action: str):
    ''' Moves the agent according to the action and outputs outcome of action '''
    if action not in shared_variables["next_valid_moves"]: 
        # if next move is not valid, reflect to agent
        return f'The current action of {action} is not valid. You must choose one action from {shared_variables["next_valid_moves"]}'
    mapping = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1), 'Stay': (0, 0)}
    
    # Retrieve from shared variables
    row, col = shared_variables["cur_pos"]
    grid = shared_variables["grid"]
    grid_size = shared_variables["grid_size"]
    known_obstacle_pos = shared_variables["known_obstacle_pos"]
    
    # Do processing for the next action
    row_offset, col_offset = mapping[action]
    newpos = (row+row_offset, col+col_offset)
    next_valid_moves = check_valid_moves(newpos, grid, grid_size)
    known_obstacle_pos = update_obstacles(newpos, grid, grid_size, known_obstacle_pos)
    
    # shift the current agent position
    grid[row][col] = ' '
    grid[row+row_offset][col+col_offset] = 'S'
    
    # Store back into shared variables
    shared_variables["cur_pos"] = newpos
    shared_variables["next_valid_moves"] = next_valid_moves
    shared_variables["known_obstacle_pos"] = known_obstacle_pos
    shared_variables["past_grid_states"].append(newpos)
    shared_variables["grid"] = grid
    
    print_grid(grid)
    
    return f'Action successful. Agent moved from {(row, col)} to {newpos}'

In [20]:
def get_global_context(agent):
    ''' Outputs additional information to the agent '''
    
    # process additional context based on shared variables (this is what is called persistent variables - variables that will be updated each step)
    global_context = f'''Agent position in grid: {agent.shared_variables["cur_pos"]}
Exit Position: {agent.shared_variables["exit_pos"]}
Last 10 Visited Grid Positions: {agent.shared_variables["past_grid_states"][:10]}
Known Obstacle Positions: {agent.shared_variables["known_obstacle_pos"]}
Next Valid Moves: {agent.shared_variables["next_valid_moves"]}'''
    
    # you can also influence how the planner performs the plan with additional details
    global_context += '''
Try to specify the action in the Subtask
Example Assigned Task: ```Navigate to (1, 1)```
Example Subtasks taking one action at a time: 'Move Down', 'Move Right' ```
'''
    return global_context

In [21]:
import random
# Customise your grid here
# O means obstacles, S means start, E means end, blank means nothing there
grid_size = 5  # Grid size
start_pos = (random.randint(0, grid_size//2 - 1), random.randint(0, grid_size//2 - 1))  # Starting position
exit_pos = (random.randint(grid_size//2, grid_size-1), random.randint(grid_size//2, grid_size-1))  # Exit position
num_obstacles = 5  # Number of obstacles

grid = generate_grid(grid_size, start_pos, exit_pos, num_obstacles)
valid_moves = check_valid_moves(start_pos, grid, grid_size)

# Assign your agent
my_agent = Agent('Maze Navigator', 
                 f'''You are to move to the Exit Position of the maze. Task is completed when Agent's position is at Exit Position.
At each step, you must output an action - Up, Down, Left, Right or Stay.
From current position (row, col), this is what you end up with after doing actions:
{{'Up': (row-1, col), 'Down': (row+1, col), 'Left': (row, col-1), 'Right': (row, col+1), 'Stay': (row, col)}}
You can only move to cells without obstacles. Only take an action from Current Valid Actions.
If your previous action is invalid, choose another action.
Top left of grid is (0, 0), bottom right is {(grid_size, grid_size)}.''', 
                 shared_variables = {
                    "cur_pos": start_pos,
                    "exit_pos": exit_pos,
                    "past_grid_states": [],
                    "next_valid_moves": valid_moves,
                    "known_obstacle_pos": [],
                    "grid_size": grid_size,
                    "grid": grid}, 
                 max_subtasks = 20,
                 get_global_context = get_global_context, # let Agent know the persistent states
                 default_to_llm = False,
                 llm = llm).assign_functions([move])

In [22]:
print('### Starting Grid ###')
print_grid(grid)
output = my_agent.run(f'Navigate to {exit_pos}')

### Starting Grid ###
S     O  
  O      
         
      E O
O       O
Observation: Agent is currently at position (0, 0) and needs to navigate to position (3, 3). The last 10 visited grid positions are empty, and there are no known obstacle positions. The next valid moves are 'Down', 'Right', and 'Stay'.
Thoughts: To reach position (3, 3), the agent needs to move strategically to avoid obstacles and reach the target efficiently. It should consider the available valid moves at each step to progress towards the exit position.
Subtask identified: Move Down to position (1, 0) to progress towards the target position (3, 3).
Calling function move with parameters {'action': 'Down'}
      O  
S O      
         
      E O
O       O
> {'output_1': 'Action successful. Agent moved from (0, 0) to (1, 0)'}

Observation: Agent successfully moved down to (1, 0) but needs to navigate to (3, 3) to complete the task.
Thoughts: The agent needs to continue moving towards the Exit Position while avoiding

In [23]:
my_agent.status()

Agent Name: Maze Navigator
Agent Description: You are to move to the Exit Position of the maze. Task is completed when Agent's position is at Exit Position.
At each step, you must output an action - Up, Down, Left, Right or Stay.
From current position (row, col), this is what you end up with after doing actions:
{'Up': (row-1, col), 'Down': (row+1, col), 'Left': (row, col-1), 'Right': (row, col+1), 'Stay': (row, col)}
You can only move to cells without obstacles. Only take an action from Current Valid Actions.
If your previous action is invalid, choose another action.
Top left of grid is (0, 0), bottom right is (5, 5).
Available Functions: ['end_task', 'move']
Shared Variables: ['cur_pos', 'exit_pos', 'past_grid_states', 'next_valid_moves', 'known_obstacle_pos', 'grid_size', 'grid', 'agent']
Task: Navigate to (3, 3)
Subtasks Completed:
Subtask: Current Results for 'progress for Navigate to (3, 3)'
Agent has successfully navigated to the Exit Position (3, 3). The last 10 visited grid po

In [24]:
# The reply to user should include everything from start position to the final position
my_agent.reply_user()

Agent has successfully navigated to the Exit Position (3, 3). The last 10 visited grid positions were [(1, 0), (2, 0), (2, 1), (3, 1), (3, 2), (3, 3)]. The known obstacle position at (3, 4) was avoided during the navigation process. The agent's current position in the grid is (3, 3).


"Agent has successfully navigated to the Exit Position (3, 3). The last 10 visited grid positions were [(1, 0), (2, 0), (2, 1), (3, 1), (3, 2), (3, 3)]. The known obstacle position at (3, 4) was avoided during the navigation process. The agent's current position in the grid is (3, 3)."

## Recommended Practice: Reset Subtasks Completed and use Global Context to convey information

- If you have multiple similar subtask names, then it is likely the Agent can be confused and think it has already done the subtask
- In this case, you can disambiguate by resetting the agent and store the persistent information in `shared_variables` and provide it to the agent using `get_global_context`
- Has the benefit of shifting the Start State closer to End State desired by resetting the Agent's planning cycle


In [25]:
def get_global_context(agent):
    ''' Outputs additional information to the agent '''
    
    # process additional context based on shared variables (this is what is called persistent variables - variables that will be updated each step)
    global_context = f'''Agent position in grid: {agent.shared_variables["cur_pos"]}
Exit Position: {agent.shared_variables["exit_pos"]}
Last 10 Visited Grid Positions: {agent.shared_variables["past_grid_states"][:10]}
Known Obstacle Positions: {agent.shared_variables["known_obstacle_pos"]}
Current Valid Actions: {agent.shared_variables["next_valid_moves"]}'''
    
    # you can also influence how the planner performs the plan with additional details
    global_context += '''
Try to specify the action in the Subtask
Example Assigned Task: ```Navigate to (1, 1)```
Example Subtasks taking one action at a time: 'Move Down', 'Move Right' ```
'''
    
    return global_context

# Customise your grid here
# O means obstacles, S means start, E means end, blank means nothing there
grid_size = 5  # Grid size
start_pos = (random.randint(0, grid_size//2 - 1), random.randint(0, grid_size//2 - 1))  # Starting position
exit_pos = (random.randint(grid_size//2, grid_size-1), random.randint(grid_size//2, grid_size-1))  # Exit position
num_obstacles = 5  # Number of obstacles

grid = generate_grid(grid_size, start_pos, exit_pos, num_obstacles)
valid_moves = check_valid_moves(start_pos, grid, grid_size)

# Assign your agent
my_agent = Agent('Maze Navigator', 
                 f'''You are to move to the Exit Position of the maze. Task is completed when Agent's position is at Exit Position.
At each step, you must output an action - Up, Down, Left, Right or Stay.
From current position (row, col), this is what you end up with after doing actions:
{{'Up': (row-1, col), 'Down': (row+1, col), 'Left': (row, col-1), 'Right': (row, col+1), 'Stay': (row, col)}}
You can only move to cells without obstacles. Only take an action from Current Valid Actions.
If your previous action is invalid, choose another action.
Top left of grid is (0, 0), bottom right is {(grid_size, grid_size)}.''', 
                 shared_variables = {
                    "cur_pos": start_pos,
                    "exit_pos": exit_pos,
                    "past_grid_states": [],
                    "next_valid_moves": valid_moves,
                    "known_obstacle_pos": [],
                    "grid_size": grid_size,
                    "grid": grid}, 
                 max_subtasks = 20,
                 get_global_context = get_global_context, # let Agent know the persistent states
                 default_to_llm = False,
                 llm = llm).assign_functions([move])

In [26]:
print('### Starting Grid ###')
print_grid(grid)

num_moves = 0
# Keep resetting subtask's history and changing start position to current position
while num_moves < 20:
    my_agent.reset()
    my_agent.run(f"Navigate to {my_agent.shared_variables['exit_pos']}", num_subtasks = 1)
    # use rule-based task checks as agent may not get it right all the time
    if my_agent.shared_variables['cur_pos'] == my_agent.shared_variables['exit_pos']: 
        my_agent.task_completed = True
        break
    num_moves += 1

### Starting Grid ###
  O   O  
O S      
    E    
  O      
      O  
Observation: Agent is currently at position (1, 1) and needs to navigate to position (2, 2). The current valid actions are Down, Right, and Stay.
Thoughts: To reach position (2, 2) from (1, 1), the agent needs to move Down and then move Right.
Subtask identified: Move Down to position (2, 1)
Calling function move with parameters {'action': 'Down'}
  O   O  
O        
  S E    
  O      
      O  
> {'output_1': 'Action successful. Agent moved from (1, 1) to (2, 1)'}

Observation: The Agent is currently at position (2, 1) and the Exit Position is at (2, 2). The Agent's last visited grid positions include (2, 1) and there is an obstacle at (3, 1). Current valid actions are Up, Left, Right, and Stay.
Thoughts: To reach the Exit Position at (2, 2) from the current position, the Agent needs to move one step to the right.
Subtask identified: Move Right
Calling function move with parameters {'action': 'Right'}
  O   O  
O

In [27]:
my_agent.status()

Agent Name: Maze Navigator
Agent Description: You are to move to the Exit Position of the maze. Task is completed when Agent's position is at Exit Position.
At each step, you must output an action - Up, Down, Left, Right or Stay.
From current position (row, col), this is what you end up with after doing actions:
{'Up': (row-1, col), 'Down': (row+1, col), 'Left': (row, col-1), 'Right': (row, col+1), 'Stay': (row, col)}
You can only move to cells without obstacles. Only take an action from Current Valid Actions.
If your previous action is invalid, choose another action.
Top left of grid is (0, 0), bottom right is (5, 5).
Available Functions: ['end_task', 'move']
Shared Variables: ['cur_pos', 'exit_pos', 'past_grid_states', 'next_valid_moves', 'known_obstacle_pos', 'grid_size', 'grid', 'agent']
Task: Navigate to (2, 2)
Subtasks Completed:
Subtask: move(action="Right")
{'output_1': 'Action successful. Agent moved from (2, 1) to (2, 2)'}

Is Task Completed: True


In [28]:
# Note Agent's task is just one step away now, since we give a new task each step
my_agent.reply_user()

Agent is already at the Exit Position (2, 2). Task is completed.


'Agent is already at the Exit Position (2, 2). Task is completed.'

# Legacy Support (Shared Variables via Function) - Calculator
- `s_` at the start of the variable names means shared variables in `Function`
    - For input, it means we take the variable from `shared_variables` instead of LLM generated input
    - For output, it means we store the variable into `shared_variables` instead of storing it in `subtasks_completed`. If `subtasks_completed` output is empty, it will be output as `{'Status': 'Completed'}`
- Example shared variables names: `s_sum`, `s_total`, `s_list_of_words`
- Generally not a preferred approach because it can be confusing to the llm how to interpret these shared variables if it is placed in the function description directly


In [29]:
# Function takes in increment (LLM generated) and s_total (retrieves from shared variable dict), and outputs to s_total (in shared variable dict)
add = Function(fn_description = "Add <increment: int> to <s_total>", 
              output_format = {"s_total": "Modified total"},
              llm = llm)

# Function takes in factor (LLM generated) and s_total (retrieves from shared variable dict), and outputs to s_total (in shared variable dict)
multiply = Function(fn_description = "Multiply <s_total> by <factor: int>",
                    output_format = {"s_total": "Modified total"},
                   llm = llm)

# Define the calculator agent and the shared_variables - Note the naming convention of s_ at the start of the names for shared variables
my_agent = Agent('Calculator', 'Does computations', 
                 shared_variables = {'s_total': 0},
                 llm = llm).assign_functions([add, multiply])

output = my_agent.run('Increment total by 3, then multiply by 5')

Observation: No subtasks have been completed yet for the Assigned Task.
Thoughts: To increment the total by 3 and then multiply by 5, we need to first add the increment and then perform the multiplication. Since the increment is a specific value, we can directly proceed with this calculation without the need for further input.
Subtask identified: Add 3 to s_total
Calling function add_increment_to_s_total with parameters {'increment': 3}
> {'Status': 'Completed'}

Observation: One subtask has been completed successfully by using the add_increment_to_s_total function.
Thoughts: To complete the remainder of the Assigned Task, we need to multiply the current total by 5.
Subtask identified: Multiply the current total by 5.
Calling function multiply_s_total with parameters {'factor': 5}
> {'Status': 'Completed'}

Observation: The subtasks to increment the total by 3 and then multiply by 5 have been successfully completed.
Thoughts: The remaining part of the assigned task is to pass the final

In [30]:
# This should be 15
print('Current Total:', my_agent.shared_variables['s_total'])

Current Total: 15
